In [1]:
# Download Entire first episode for transcription test
import requests

episode_link = 'https://audio.podigee-cdn.net/1038341-m-1864d1131f1e7fdf980b1a120d7336c1.mp3?source=feed'
episode_file = requests.get(episode_link, allow_redirects=True)
open('GAG388.mp3', 'wb').write(episode_file.content)

50709714

In [16]:
# Cut out first 60 seconds as fast sample
from pydub import AudioSegment

sound = AudioSegment.from_mp3("GAG001.mp3")
sound.set_channels(1)
sound.set_frame_rate(16000)

#Selecting Portion we want to cut
StrtMin = 0
StrtSec = 0
EndMin = 0
EndSec = 60

# Time to milliseconds conversion
StrtTime = StrtMin*60*1000+StrtSec*1000
EndTime = StrtMin*60*1000+EndSec*1000
# Opening file and extracting portion of it
extract = sound[StrtTime:EndTime]
# Saving file in required location
extract.export("GAG001_sample.mp3", format="mp3")

<_io.BufferedRandom name='GAG001_sample.mp3'>

In [9]:
import requests
import os

API_TOKEN = os.environ['HF_TOKEN']
API_URL = "https://api-inference.huggingface.co/models/bofenghuang/whisper-small-cv11-german"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response

output = query("GAG001.mp3")

In [7]:
output

{'text': "Hallo, liebe Hörerinnen und Hörer, wir melden uns kurz aus der Zukunft. Seit Folge zweihundertsitzig heißt dieser Podcast Geschichten aus der Geschichte. Mehr dazu findet ihr unter geschichte.fm. Und jetzt geht's zurück in die Vergangenheit viel Spaß."}

In [ ]:
import torch

from transformers import pipeline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load pipeline
pipe = pipeline("automatic-speech-recognition", model="bofenghuang/whisper-small-cv11-german",chunk_length_s=30, device=device)

In [3]:
result = pipe('GAG001.mp3', return_timestamps=True)
result

/Users/marcgehrke/miniconda3/envs/kag_airflow/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': "Hallo, liebe Hörerinnen und Hörer, wir melden uns kurz aus der Zukunft. Seit Folge zweihundertsitzig heißt dieser Podcast Geschichten aus der Geschichte. Mehr dazu findet ihr unter Geschichte.fm. Und jetzt geht's zurück in die Vergangenheit viel Spaß.Werden ein bisschen Geschichte, dann werden Sainte Daimorta wieder sich damals entwicklen können. Hallo und willkommen zu Zeitsprung Geschichten aus der Geschichte Ich bin Daniel und ich bin Richard. Und bei Zeitsprungen präsentieren wir euch vergessehene eigenes Waschen-Ereignis und Zusammenhänge der Geschichte in kurzen Episoden häufig mit ein bisschen Augenzwingern, aber immer quälner recherchiert. Und du, Richard, hast du eine Geschichte mitgebracht? Ja, ich habe eine Geschichte mitgebracht. Findest du diese sehr gute Anfangsgeschichte, weil sie uns gleich einmal sehr, sehr tief in die Geschichte eintauchen lässt, und zwar ins frühe Mittelalter. Frühes Mittelalter heißt, in dem Fall sechstes Jahrhundert nach Christus, und der

In [5]:
with open('GAG001_transcript.txt', 'w') as f:
    f.write(result['text'])

In [ ]:
import openai

audio_file= open("GAG001_sample.mp3", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)